# Action Detection

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import numpy as np
from scipy.io import loadmat
from random import shuffle
from torch.utils.data import Dataset, DataLoader
import cv2
import os
from sklearn.model_selection import train_test_split
import re

## Data Loader

In [99]:
video_rootdir="./ReCompress_Videos"
mask_rootdir="./puppet_mask"
pose_rootdir="./joint_positions"

video_pathes=[]
mask_pathes=[]
pose_pathes=[]
for root, dirs, files in os.walk(video_rootdir):
    for file in files:
        if file[0].startswith(".") or root.endswith('.AppleDouble'):
            continue
        video_pathes.append(os.path.join(root, file))

for root, dirs, files in os.walk(mask_rootdir):
    for file in files:
        if file[0].startswith(".") or root.endswith('.AppleDouble'):
            continue
        mask_pathes.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(pose_rootdir):
    for file in files:
        if file[0].startswith(".") or root.endswith('.AppleDouble'):
            continue
        pose_pathes.append(os.path.join(root, file))         

video_pathes = sorted(video_pathes)
mask_pathes = sorted(mask_pathes)
pose_pathes = sorted(pose_pathes)

video_list = [re.findall(r"[\w_]+\d+", x)[0] for x in video_pathes]
mask_list = [re.findall(r"[\w_]+\d+", x)[0] for x in mask_pathes]
pose_list = [re.findall(r"[\w_]+\d+", x)[0] for x in pose_pathes]

# make sure every thing matches
for x, y, z in zip(video_list, mask_list, pose_list):
    assert (x == y) and (x == z)
    


In [19]:
import re

In [105]:
video_pathes = sorted(video_pathes)
mask_pathes = sorted(mask_pathes)
pose_pathes = sorted(pose_pathes)

In [106]:
video_list = [re.findall(r"[\w_]+\d+", x)[0] for x in video_pathes]
print(video_list)

['April_09_brush_hair_u_nm_np1_ba_goo_0', 'April_09_brush_hair_u_nm_np1_ba_goo_1', 'April_09_brush_hair_u_nm_np1_ba_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0', 'Brushing_Hair_with_Beth_brush_hair_h_nm_np1_le_goo_0', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_0', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_1', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2', '_December_2008_brush_hair_u_cm_np1_ba_goo_1', '_December_2008_brush_hair_u_cm_np1_ba_goo_2', '_December_2008_brush_hai

In [107]:
mask_list = [re.findall(r"[\w_]+\d+", x)[0] for x in mask_pathes]
print(mask_list)

['April_09_brush_hair_u_nm_np1_ba_goo_0', 'April_09_brush_hair_u_nm_np1_ba_goo_1', 'April_09_brush_hair_u_nm_np1_ba_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0', 'Brushing_Hair_with_Beth_brush_hair_h_nm_np1_le_goo_0', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_0', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_1', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2', '_December_2008_brush_hair_u_cm_np1_ba_goo_1', '_December_2008_brush_hair_u_cm_np1_ba_goo_2', '_December_2008_brush_hai

In [108]:
pose_list = [re.findall(r"[\w_]+\d+", x)[0] for x in pose_pathes]
print(pose_list)

['April_09_brush_hair_u_nm_np1_ba_goo_0', 'April_09_brush_hair_u_nm_np1_ba_goo_1', 'April_09_brush_hair_u_nm_np1_ba_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2', 'Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0', 'Brushing_Hair_with_Beth_brush_hair_h_nm_np1_le_goo_0', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_0', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_1', 'Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_2', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_1', 'Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2', '_December_2008_brush_hair_u_cm_np1_ba_goo_1', '_December_2008_brush_hair_u_cm_np1_ba_goo_2', '_December_2008_brush_hai

In [110]:
# make sure every thing matches
for x, y, z in zip(video_list, mask_list, pose_list):
    assert (x == y) and (x == z)

In [84]:
print(len(video_pathes))

928


In [83]:
print(video_pathes)

['./ReCompress_Videos/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.avi', './ReCompress_Videos/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.avi', './ReCompress_Videos/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.avi', './ReCompress_Videos/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3.avi', './ReCompress_Videos/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1.avi', './ReCompress_Videos/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2.avi', './ReCompress_Videos/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0.avi', './ReCompress_Videos/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0.avi', './ReCompress_Videos/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_2.avi', './ReCompress_Videos/brush_hair/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_0.avi', './ReCompress_Videos/brush_hair/Brushing_hair__th

In [5]:
print(mask_pathes)

['./puppet_mask/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0/puppet_mask.mat', './puppet_mask/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1/puppet_mask.mat', './puppet_mask/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2/puppet_mask.mat', './puppet_mask/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3/puppet_mask.mat', './puppet_mask/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1/puppet_mask.mat', './puppet_mask/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2/puppet_mask.mat', './puppet_mask/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0/puppet_mask.mat', './puppet_mask/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0/puppet_mask.mat', './puppet_mask/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_2/puppet_mask.mat', './puppet_mask/brush_hair/Brushing_hair__the_right_way_brush_hair_u_nm_np1_fr_goo_0/puppet_

In [6]:
print(pose_pathes)

['./joint_positions/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0/joint_positions.mat', './joint_positions/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1/joint_positions.mat', './joint_positions/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2/joint_positions.mat', './joint_positions/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3/joint_positions.mat', './joint_positions/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1/joint_positions.mat', './joint_positions/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2/joint_positions.mat', './joint_positions/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0/joint_positions.mat', './joint_positions/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_0/joint_positions.mat', './joint_positions/brush_hair/brush_my_hair_without_wearing_the_glasses_brush_hair_u_nm_np1_fr_goo_2/joint_positions.mat', './joint_positions/

In [32]:
from sklearn.model_selection import train_test_split

In [85]:
video_pathes_train, video_pathes_valid, mask_pathes_train, mask_pathes_valid, pose_pathes_train, pose_pathes_valid = \
    train_test_split(video_pathes, mask_pathes, pose_pathes, test_size=0.1)
class_names=[name for name in os.listdir(video_rootdir) if not name.startswith(".")]


In [91]:
class JHMDB(torch.utils.data.Dataset):
    def __init__(self, video_pathes, mask_pathes, pose_pathes, class_names):
        
        self.data = {'video': [], 'label': [], 'mask':[], 'pose':[], 'scale':[]}
        
        self.classdict = {}
        for i, x in enumerate(class_names):
            self.classdict[x] = i

        video_num=len(video_pathes)
        mask_num=len(mask_pathes)

        for i in range(video_num):
            video=[]
            cap = cv2.VideoCapture(video_pathes[i])
            has_frame=True
            while(has_frame):
                _, frame = cap.read()
                has_frame = frame is not None

                if has_frame:
                    frame = cv2.resize(frame, (112, 112), interpolation = cv2.INTER_CUBIC) # (112, 112, 3)
                    #frame = np.transpose(frame, (2, 0, 1)) # (3, 112, 112)
                    video.append(frame)
            cap.release()
            self.data['video'].append(video)

            mask_mat = loadmat(mask_pathes[i]) 
            masks = cv2.resize(mask_mat['part_mask'], (112, 112), interpolation = cv2.INTER_CUBIC) # (112, 112, F)
            self.data['mask'].append(masks)
            self.data['label'].append(video_pathes[i].split('/')[-2])
            pose_mat = loadmat(pose_pathes[i])['pos_img']
            scale = loadmat(pose_pathes[i])['scale']
            self.data['pose'].append(pose_mat)
            self.data['scale'].append(scale[0]) # redundant dim
            
            
    def _compute_mean(self):
        meanstd_file = './data/jhmdbmean'
        if os.path.isfile(meanstd_file):
            meanstd = torch.load(meanstd_file)
        else:
            #mean = torch.zeros(3)
            #std = torch.zeros(3)
            mean = np.zeros(3)
            std = np.zeros(3)
            cnt = 0
            for videos in self.data['video']:
                for img in videos:
                    # CxHxW
                    mean += np.reshape(img, (-1, img.shape[-1])).mean(0)
                    std += np.reshape(img, (-1, img.shape[-1])).std(0) 
                    cnt += 1
            mean /= cnt
            std /= cnt
            mean = torch.FloatTensor(mean)
            std = torch.FloatTensor(std)
            meanstd = {
                'mean': mean,
                'std': std,
                }
            torch.save(meanstd, meanstd_file)
            
        print('    Mean: %.4f, %.4f, %.4f' % (meanstd['mean'][0], meanstd['mean'][1], meanstd['mean'][2]))
        print('    Std:  %.4f, %.4f, %.4f' % (meanstd['std'][0], meanstd['std'][1], meanstd['std'][2]))
            
        return meanstd['mean'], meanstd['std']
    
    def __getitem__(self, index):
        # video (F, C, 112, 112) to be reshaped on the fly
        # label scala, 
        # mask (112, 112, F)
        # pose (2, 15, F)
        # scale (F)
        
        frame_num = self.data['mask'][index].shape[2]
        F = 15
        # randomly select 15 consecutive frames (F = 15)
        #start_frame = np.random.randint(0, high=frame_num-F+1)
        
        # randomly select 15 frames
        frame_shuffle=list(range(frame_num))
        shuffle(frame_shuffle)
        selected_frame=sorted(frame_shuffle[0:F])
        pose_data = torch.from_numpy(self.data['pose'][index][:,:,selected_frame].astype('float'))
        
        # change pose position according to resize
        pose_data = torch.from_numpy(self.data['pose'][index][:,:,start_frame:start_frame+F].astype('float'))
        pose_data[0,:,:] = pose_data[0,:,:] * 112 / 240
        pose_data[1,:,:] = pose_data[1,:,:] * 112 / 320
        
        return torch.from_numpy(np.array(self.data['video'][index][start_frame:start_frame+F])).int(), \
            torch.LongTensor([self.classdict[self.data['label'][index]]]), \
            torch.from_numpy(self.data['mask'][index][:,:,start_frame:start_frame+F].astype('float')), \
            pose_data, \
            torch.from_numpy(self.data['scale'][index][start_frame:start_frame+F].astype('float'))
        
    def __len__(self):
        return len(self.data['scale'])

In [435]:
valid_dataset.data['video']

[[array([[[  5,  14,   8],
          [ 10,  22,  18],
          [  9,  24,  20],
          ...,
          [  9,  20,  15],
          [  7,  17,  14],
          [  9,  20,  14]],
  
         [[  8,  17,  11],
          [ 30,  42,  38],
          [ 14,  29,  24],
          ...,
          [ 13,  24,  18],
          [ 11,  20,  18],
          [ 11,  22,  16]],
  
         [[ 10,  21,  15],
          [ 39,  51,  47],
          [ 14,  31,  27],
          ...,
          [ 25,  36,  30],
          [ 15,  26,  20],
          [ 14,  25,  19]],
  
         ...,
  
         [[ 63,  78,  60],
          [ 99, 119, 100],
          [ 92, 112,  93],
          ...,
          [ 20,  39,  30],
          [ 26,  42,  33],
          [ 30,  44,  36]],
  
         [[ 45,  56,  42],
          [108, 123, 109],
          [ 89, 109,  90],
          ...,
          [ 18,  31,  24],
          [ 17,  30,  22],
          [ 12,  23,  18]],
  
         [[ 22,  28,  17],
          [ 59,  72,  59],
          [ 48,  60,  46

In [434]:
len(valid_dataset.data['video'])

10

In [24]:
train_dataset = JHMDB(video_pathes_train, mask_pathes_train, pose_pathes_train, class_names)


In [92]:
valid_dataset = JHMDB(video_pathes_valid, mask_pathes_valid, pose_pathes_valid, class_names)


93


In [93]:
valid_dataset._compute_mean()

(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 223 213]
 [203 197 187]
 [218 212 202]
 ...
 [ 96  97  92]
 [ 97  98  93]
 [ 98  99  94]]
(112, 112, 3)
(12544, 3)
[[229 2

(112, 112, 3)
(12544, 3)
[[ 0  5  4]
 [ 7 15 14]
 [14 24 23]
 ...
 [28 42 29]
 [ 5 18  7]
 [15 27 12]]
(112, 112, 3)
(12544, 3)
[[ 0  8  7]
 [12 22 21]
 [11 21 20]
 ...
 [39 53 39]
 [ 2 14  3]
 [ 8 19  8]]
(112, 112, 3)
(12544, 3)
[[ 1 11 10]
 [ 0  8  7]
 [14 24 23]
 ...
 [40 53 42]
 [39 51 43]
 [30 40 32]]
(112, 112, 3)
(12544, 3)
[[18 28 27]
 [13 23 22]
 [14 24 23]
 ...
 [36 49 38]
 [26 38 30]
 [30 41 35]]
(112, 112, 3)
(12544, 3)
[[18 28 27]
 [13 23 22]
 [15 25 24]
 ...
 [36 49 38]
 [26 38 30]
 [30 41 35]]
(112, 112, 3)
(12544, 3)
[[13 23 22]
 [13 23 22]
 [ 8 18 17]
 ...
 [14 26 18]
 [42 55 44]
 [ 0  8  0]]
(112, 112, 3)
(12544, 3)
[[11 21 20]
 [10 20 19]
 [ 9 19 18]
 ...
 [ 8 20 12]
 [28 41 32]
 [18 31 20]]
(112, 112, 3)
(12544, 3)
[[ 1 11 10]
 [ 2 12 11]
 [ 4 14 13]
 ...
 [ 8 17 15]
 [10 22 14]
 [18 30 22]]
(112, 112, 3)
(12544, 3)
[[149 122  99]
 [149 122  99]
 [149 122  99]
 ...
 [ 79  93  73]
 [ 82  93  74]
 [ 83  93  74]]
(112, 112, 3)
(12544, 3)
[[151 121  99]
 [151 121  99]


(112, 112, 3)
(12544, 3)
[[22 21 19]
 [14 17 15]
 [20 23 21]
 ...
 [11 14 12]
 [ 6 12 11]
 [12 12 10]]
(112, 112, 3)
(12544, 3)
[[21 20 20]
 [18 17 16]
 [23 21 17]
 ...
 [10 12 11]
 [ 8 11 10]
 [ 9 12 10]]
(112, 112, 3)
(12544, 3)
[[17 19 21]
 [14 18 16]
 [18 22 17]
 ...
 [ 5 13 10]
 [ 4 11 10]
 [ 8 11 11]]
(112, 112, 3)
(12544, 3)
[[21 19 21]
 [14 18 16]
 [16 22 17]
 ...
 [ 5 13 10]
 [ 4 10 11]
 [ 8 11  9]]
(112, 112, 3)
(12544, 3)
[[20 16 19]
 [ 6 16 13]
 [18 19 13]
 ...
 [ 2 10  9]
 [ 3  9 10]
 [ 9 13  8]]
(112, 112, 3)
(12544, 3)
[[20 16 19]
 [ 6 16 13]
 [18 19 13]
 ...
 [ 2 10  9]
 [ 3  9 10]
 [ 9 13  8]]
(112, 112, 3)
(12544, 3)
[[19 17 19]
 [ 5 17 12]
 [18 21 17]
 ...
 [ 0  9  6]
 [ 1  9  8]
 [11 12  7]]
(112, 112, 3)
(12544, 3)
[[20 19 19]
 [ 4 16 11]
 [18 20 19]
 ...
 [ 0  9  6]
 [ 1  9  8]
 [11 12  7]]
(112, 112, 3)
(12544, 3)
[[17 17 15]
 [ 4 17 10]
 [20 23 19]
 ...
 [ 3 13 10]
 [ 5 12 11]
 [13 14  8]]
(112, 112, 3)
(12544, 3)
[[17 17 15]
 [ 4 17 10]
 [20 23 19]
 ...
 [ 3 13

(112, 112, 3)
(12544, 3)
[[ 9 24  6]
 [ 9 23  8]
 [10 24 11]
 ...
 [57 65 70]
 [51 59 64]
 [50 58 62]]
(112, 112, 3)
(12544, 3)
[[ 9 24  6]
 [ 9 23  8]
 [ 9 23 10]
 ...
 [51 59 63]
 [46 54 59]
 [57 65 70]]
(112, 112, 3)
(12544, 3)
[[ 9 23  8]
 [ 9 23  8]
 [10 24  9]
 ...
 [46 54 58]
 [50 58 63]
 [64 72 76]]
(112, 112, 3)
(12544, 3)
[[ 9 23  8]
 [ 9 23  8]
 [12 26 13]
 ...
 [47 55 59]
 [40 48 52]
 [38 46 50]]
(112, 112, 3)
(12544, 3)
[[ 9 23 10]
 [ 9 23  8]
 [13 26 16]
 ...
 [47 57 61]
 [25 35 39]
 [23 33 37]]
(112, 112, 3)
(12544, 3)
[[11 21 13]
 [13 24 13]
 [15 25 17]
 ...
 [52 65 68]
 [37 50 53]
 [35 48 51]]
(112, 112, 3)
(12544, 3)
[[11 21 13]
 [15 26 15]
 [17 28 17]
 ...
 [47 60 63]
 [41 54 57]
 [30 43 46]]
(112, 112, 3)
(12544, 3)
[[10 22 14]
 [11 23 15]
 [13 25 17]
 ...
 [48 61 64]
 [37 50 53]
 [23 36 39]]
(112, 112, 3)
(12544, 3)
[[10 22 14]
 [11 23 15]
 [12 24 16]
 ...
 [36 48 52]
 [31 44 47]
 [24 37 40]]
(112, 112, 3)
(12544, 3)
[[10 22 14]
 [11 23 15]
 [12 23 17]
 ...
 [32 42

(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 63 126 126]
 [ 70 130 130]
 [ 73 130 132]]
(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 63 126 126]
 [ 69 129 129]
 [ 73 131 133]]
(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 63 126 126]
 [ 69 129 129]
 [ 73 131 133]]
(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 63 126 126]
 [ 69 129 129]
 [ 73 131 133]]
(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 62 125 125]
 [ 68 128 129]
 [ 74 131 133]]
(112, 112, 3)
(12544, 3)
[[235 241 222]
 [235 241 222]
 [235 241 222]
 ...
 [ 64 127 127]
 [ 68 128 129]
 [ 72 130 132]]
(112, 112, 3)
(12544, 3)
[[134 133 122]
 [143 145 129]
 [140 145 129]
 ...
 [249 218 177]
 [245 208 176]
 [241 205 178]]
(112, 112, 3)
(12544, 3)
[[134 133 122]
 [143 145 129]
 [140 145 129]
 ...
 [249 217 177]
 [246 209 177]
 [240 204 177]]
(112, 112, 3)
(12544, 3)
[[132 1

...

(112, 112, 3)
(12544, 3)
[[246 245 247]
 [250 249 251]
 [245 244 246]
 ...
 [ 84 136 110]
 [ 83 135 109]
 [ 83 135 109]]
(112, 112, 3)
(12544, 3)
[[249 249 249]
 [248 246 248]
 [249 247 249]
 ...
 [ 83 140 113]
 [ 81 138 111]
 [ 81 138 111]]
(112, 112, 3)
(12544, 3)
[[250 249 249]
 [248 246 248]
 [244 243 244]
 ...
 [ 87 144 117]
 [ 86 143 116]
 [ 85 142 115]]
(112, 112, 3)
(12544, 3)
[[251 249 252]
 [247 246 248]
 [246 244 249]
 ...
 [ 90 144 118]
 [ 90 145 119]
 [ 89 143 117]]
(112, 112, 3)
(12544, 3)
[[245 244 246]
 [247 246 248]
 [250 247 253]
 ...
 [ 87 141 115]
 [ 88 142 116]
 [ 88 142 116]]
(112, 112, 3)
(12544, 3)
[[242 241 244]
 [250 248 253]
 [247 245 251]
 ...
 [ 80 135 109]
 [ 80 134 108]
 [ 87 141 115]]
(112, 112, 3)
(12544, 3)
[[243 242 245]
 [250 248 251]
 [245 244 246]
 ...
 [ 79 133 107]
 [ 77 131 105]
 [ 84 138 112]]
(112, 112, 3)
(12544, 3)
[[244 243 246]
 [251 250 252]
 [247 246 248]
 ...
 [ 72 124  98]
 [ 71 123  97]
 [ 76 128 102]]
(112, 112, 3)
(12544, 3)
[[248 2

(112, 112, 3)
(12544, 3)
[[246 247 249]
 [245 247 249]
 [249 247 249]
 ...
 [ 34  72  94]
 [ 32  70  92]
 [ 38  76  97]]
(112, 112, 3)
(12544, 3)
[[248 245 251]
 [247 244 250]
 [245 242 248]
 ...
 [ 35  72  96]
 [ 31  68  92]
 [ 34  71  95]]
(112, 112, 3)
(12544, 3)
[[243 248 246]
 [245 248 248]
 [245 247 249]
 ...
 [ 35  70  99]
 [ 31  66  95]
 [ 34  69  98]]
(112, 112, 3)
(12544, 3)
[[248 249 244]
 [248 249 244]
 [248 249 244]
 ...
 [ 37  72 101]
 [ 35  70  99]
 [ 34  69  98]]
(112, 112, 3)
(12544, 3)
[[13 17  3]
 [21 24  3]
 [26 24  1]
 ...
 [ 7 10  0]
 [10 13  3]
 [ 8 10  3]]
(112, 112, 3)
(12544, 3)
[[12 16  2]
 [20 23  1]
 [23 20  0]
 ...
 [ 9 12  2]
 [ 8 11  1]
 [ 8 10  3]]
(112, 112, 3)
(12544, 3)
[[21 20  7]
 [28 24  3]
 [30 21  0]
 ...
 [ 9 12  2]
 [ 8 10  3]
 [ 7  9  2]]
(112, 112, 3)
(12544, 3)
[[28 20  8]
 [33 25  5]
 [29 19  0]
 ...
 [ 8 11  1]
 [ 6  8  1]
 [ 6  8  1]]
(112, 112, 3)
(12544, 3)
[[37 26 18]
 [40 27 10]
 [37 21  1]
 ...
 [ 8 11  1]
 [ 6  8  1]
 [ 6  8  1]]
(

(112, 112, 3)
(12544, 3)
[[148 173 193]
 [171 195 219]
 [162 184 216]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[148 173 193]
 [171 195 219]
 [163 185 216]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[147 173 192]
 [171 195 219]
 [164 187 218]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[159 185 206]
 [170 192 220]
 [166 191 224]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[158 184 206]
 [170 193 221]
 [166 191 225]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[156 183 208]
 [168 192 222]
 [164 191 227]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[156 183 208]
 [168 192 222]
 [164 191 227]
 ...
 [151 151 151]
 [151 151 151]
 [151 151 151]]
(112, 112, 3)
(12544, 3)
[[161 188 213]
 [167 190 220]
 [162 189 224]
 ...
 [147 146 148]
 [147 146 148]
 [147 146 148]]
(112, 112, 3)
(12544, 3)
[[161 1

(112, 112, 3)
(12544, 3)
[[48 53 46]
 [51 56 49]
 [54 59 52]
 ...
 [79 76 55]
 [40 35 21]
 [37 33 23]]
(112, 112, 3)
(12544, 3)
[[129 129 129]
 [112 112 112]
 [ 95  95  95]
 ...
 [ 64  75  66]
 [ 71  81  75]
 [ 80  77  76]]
(112, 112, 3)
(12544, 3)
[[129 129 129]
 [112 112 112]
 [ 95  95  95]
 ...
 [ 64  75  66]
 [ 71  81  75]
 [ 80  77  76]]
(112, 112, 3)
(12544, 3)
[[126 126 126]
 [104 104 104]
 [ 97  97  97]
 ...
 [ 64  72  65]
 [ 70  79  73]
 [ 80  79  77]]
(112, 112, 3)
(12544, 3)
[[117 117 117]
 [ 99  99  99]
 [103 103 103]
 ...
 [ 65  79  67]
 [ 67  78  72]
 [ 77  80  80]]
(112, 112, 3)
(12544, 3)
[[118 118 118]
 [100 100 100]
 [104 104 104]
 ...
 [ 68  80  67]
 [ 67  79  69]
 [ 75  81  76]]
(112, 112, 3)
(12544, 3)
[[125 125 125]
 [ 99  99  99]
 [109 109 109]
 ...
 [ 65  78  67]
 [ 71  80  72]
 [ 83  83  81]]
(112, 112, 3)
(12544, 3)
[[127 127 127]
 [ 99  99  99]
 [110 110 110]
 ...
 [ 67  81  66]
 [ 71  82  70]
 [ 85  83  86]]
(112, 112, 3)
(12544, 3)
[[129 129 129]
 [ 98  98 

(
   93.3131
  107.5407
  105.2729
 [torch.FloatTensor of size 3], 
  56.1015
  54.7933
  55.3897
 [torch.FloatTensor of size 3])

In [404]:
i = 0
for x in valid_dataset:
    #print(x)
    #if i == 100:
    #print(x)
    break
    

pose0 
(0 ,.,.) = 

Columns 0 to 6 
    73.2988   73.1375   75.1579   75.1575   85.1226   85.1210   81.2326
   85.9737   82.8679   87.3984   87.3984   85.7549   85.7581   76.3919
   78.9094   78.9198   77.2249   77.2266   80.0737   80.0678   74.1840
   89.2000   89.7714   89.7714   89.7714  100.6286  100.6286   99.4857
   52.5143   52.5143   52.5143   52.5143   59.3714   64.5143   64.5143
  100.9143  100.9143  103.2000  103.2000  103.2000  100.3429   93.4858
   77.0857   79.3711   81.6572   81.6572   81.0857   81.0857   73.0857
   79.0161   71.7272   70.5931   70.5938   99.8437   99.8411  115.2831
   35.4909   32.6325   32.5019   32.4997   39.5206   52.0288   55.4611
  108.5010  114.2172  121.6837  121.6854  127.5623  128.8135  122.5601
   59.1240   60.3645   59.3239   59.3240   65.2265   68.6559   73.5048
   36.5838   35.3158   32.9698   32.9401   68.6881   82.3877   89.8667
   33.0639   30.7856   24.8418   24.8338   69.0412   76.1817   87.1440
  133.4709  135.1547  136.1002  136.0993

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=10, shuffle=True)
    #num_workers=2, pin_memory=True) 

In [409]:
valid_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=5, shuffle=True)
    #num_workers=2, pin_memory=True)

In [410]:
for x in valid_loader:
    # transpose 
    print(x[0].transpose(4,3).transpose(3,2).transpose(2,1).size())
    for t in x[1:]:
        print(t.size())
    
    break

torch.Size([5, 3, 15, 112, 112])
torch.Size([5, 1])
torch.Size([5, 112, 112, 15])
torch.Size([5, 2, 15, 15])
torch.Size([5, 15])


## Model

In [0]:
class C3D(nn.Module):
    """
    The C3D network as described in [1].
    """

    def __init__(self):
        super(C3D, self).__init__()

        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 21)

        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, x):

        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192)
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)

        return logits